In [6]:
import polars as pl
import numpy as np

In [7]:
df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "A": [True, True, False, False, False],
    }
)
print(df)

shape: (5, 4)
┌──────┬───────┬──────────┬───────┐
│ nrs  ┆ names ┆ random   ┆ A     │
│ ---  ┆ ---   ┆ ---      ┆ ---   │
│ i64  ┆ str   ┆ f64      ┆ bool  │
╞══════╪═══════╪══════════╪═══════╡
│ 1    ┆ foo   ┆ 0.427718 ┆ true  │
│ 2    ┆ ham   ┆ 0.519864 ┆ true  │
│ 3    ┆ spam  ┆ 0.717269 ┆ false │
│ null ┆ egg   ┆ 0.972496 ┆ false │
│ 5    ┆ null  ┆ 0.939138 ┆ false │
└──────┴───────┴──────────┴───────┘


In [8]:
# count number of rows where A == True
df.select(pl.col("A").sum() / pl.len()).item()

0.4

In [9]:
# different ways to index a single element of a Polars dataframe
irow = 3
assert df[irow, 'names'] == df.item(irow, 'names')